<a href="https://colab.research.google.com/github/yk-Jeong/dsnote_examples/blob/main/dsnote_11_lightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###LightGBM 실습

**모델 설명**

- XGBoost 이후로 나온 최신 부스팅 모델
- 장점: 빠르고 높은 정확도, 예측에 영향을 미친 변수의 중요도를 확인할 수 있음, **변수가 많고 데이터가 클수록** 상대적으로 뛰어난 성능을 보임, 종속변수의 범주형에 무관하게 사용 가능 
- 단점: 복잡한 모델이어서 해석이 어려움, 하이퍼파라미터 튜닝이 까다로움



** 데이터 설명**

- 사용할 데이터 세트:  카드 거래 내역
- 목표: 이상 거래 예측 

1. lib and data load 

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
file_url = "https://media.githubusercontent.com/media/musthave-ML10/data_source/main/fraud.csv"
data = pd.read_csv(file_url) 

In [37]:
data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 22 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   trans_date_trans_time  object 
 1   cc_num                 int64  
 2   merchant               object 
 3   category               object 
 4   amt                    float64
 5   first                  object 
 6   last                   object 
 7   gender                 object 
 8   street                 object 
 9   city                   object 
 10  state                  object 
 11  zip                    int64  
 12  lat                    float64
 13  long                   float64
 14  city_pop               int64  
 15  job                    object 
 16  dob                    object 
 17  trans_num              object 
 18  unix_time              int64  
 19  merch_lat              float64
 20  merch_long             float64
 21  is_fraud               int64  
dtypes: float64(5), int

show_counts로 non-null한 값의 수를 확인

In [39]:
data.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1852394 non-null  object 
 1   cc_num                 1852394 non-null  int64  
 2   merchant               1852394 non-null  object 
 3   category               1852394 non-null  object 
 4   amt                    1852394 non-null  float64
 5   first                  1852394 non-null  object 
 6   last                   1852394 non-null  object 
 7   gender                 1852394 non-null  object 
 8   street                 1852394 non-null  object 
 9   city                   1852394 non-null  object 
 10  state                  1852394 non-null  object 
 11  zip                    1852394 non-null  int64  
 12  lat                    1852394 non-null  float64
 13  long                   1852394 non-null  float64
 14  city_pop          

In [40]:
round(data.describe(), 2)

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1.852394e+06,1852394.00,1852394.00,1852394.00,1852394.00,1852394.00,1.852394e+06,1852394.00,1852394.00,1852394.00
mean,4.173860e+17,70.06,48813.26,38.54,-90.23,88643.67,1.358674e+09,38.54,-90.23,0.01
std,1.309115e+18,159.25,26881.85,5.07,13.75,301487.62,1.819508e+07,5.11,13.76,0.07
min,6.041621e+10,1.00,1257.00,20.03,-165.67,23.00,1.325376e+09,19.03,-166.67,0.00
25%,1.800429e+14,9.64,26237.00,34.67,-96.80,741.00,1.343017e+09,34.74,-96.90,0.00
50%,3.521417e+15,47.45,48174.00,39.35,-87.48,2443.00,1.357089e+09,39.37,-87.44,0.00
75%,4.642255e+15,83.10,72042.00,41.94,-80.16,20328.00,1.374581e+09,41.96,-80.25,0.00
max,4.992346e+18,28948.90,99921.00,66.69,-67.95,2906700.00,1.388534e+09,67.51,-66.95,1.00


is_fraud의 mean이 0.01로, 데이터가 매우 imbalance함을 확인할 수 있음

2. data cleaning

- 불필요한 변수를 제외하기 위해 명확한 기준을 세울 것. 

(참고: 이상거래 탐지의 기본 아이디어는 '해당 고객의 기존 거래 패턴에서 벗어나는 경우'를 감지하는 것)

In [41]:
# 우선은 예제 작성자가 불필요하다고 판단한 변수를 따라보기로 함 

data.drop(['first', 'last', 'street', 'city', 'state', 'zip', 'trans_num', 'unix_time', 'job', 'merchant'], axis=1, inplace=True)

In [42]:
# trans_date_trans_time 변수를 datetime 형으로 변환 

data['trans_data_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])

In [43]:
data

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,trans_data_trans_time
0,2019-01-01 00:00:18,2703186189652095,misc_net,4.97,F,36.0788,-81.1781,3495,1988-03-09,36.011293,-82.048315,0,2019-01-01 00:00:18
1,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,2019-01-01 00:00:44
2,2019-01-01 00:00:51,38859492057661,entertainment,220.11,M,42.1808,-112.2620,4154,1962-01-19,43.150704,-112.154481,0,2019-01-01 00:00:51
3,2019-01-01 00:01:16,3534093764340240,gas_transport,45.00,M,46.2306,-112.1138,1939,1967-01-12,47.034331,-112.561071,0,2019-01-01 00:01:16
4,2019-01-01 00:03:06,375534208663984,misc_pos,41.96,M,38.4207,-79.4629,99,1986-03-28,38.674999,-78.632459,0,2019-01-01 00:03:06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852389,2020-12-31 23:59:07,30560609640617,health_fitness,43.77,M,40.4931,-91.8912,519,1966-02-13,39.946837,-91.333331,0,2020-12-31 23:59:07
1852390,2020-12-31 23:59:09,3556613125071656,kids_pets,111.84,M,29.0393,-95.4401,28739,1999-12-27,29.661049,-96.186633,0,2020-12-31 23:59:09
1852391,2020-12-31 23:59:15,6011724471098086,kids_pets,86.88,F,46.1966,-118.9017,3684,1981-11-29,46.658340,-119.715054,0,2020-12-31 23:59:15
1852392,2020-12-31 23:59:24,4079773899158,travel,7.99,M,44.6255,-116.4493,129,1965-12-15,44.470525,-117.080888,0,2020-12-31 23:59:24


3. feature engineering 

1) 개인의 거래 내역 특징을 이해하기 위해 z-score를 활용하기로 함 (cc_num을 기준으로 groupby()를 사용)

In [44]:
amt_info = data.groupby('cc_num').agg(['mean', 'std'])['amt'].reset_index()
amt_info.head()

,cc_num,mean,std
0,60416207185,59.257796,142.869746
1,60422928733,65.483159,92.042844
2,60423098130,96.376084,1000.693872
3,60427851591,107.487550,131.014534
4,60487002085,64.096925,153.207660


In [45]:
data = data.merge(amt_info, on='cc_num', how = 'left')

In [46]:
# z-score 계산 
data['amt_z_score'] = (data['amt'] - data['mean']) / data['std']

In [47]:
data[['amt', 'mean', 'std', 'amt_z_score']].head()

,amt,mean,std,amt_z_score
0,4.97,89.408743,127.530101,-0.662108
1,107.23,56.078113,159.201852,0.321302
2,220.11,69.924272,116.688602,1.287064
3,45.00,80.090040,280.077880,-0.125287
4,41.96,95.341146,94.322842,-0.565941


In [48]:
# mean, std 제거: z-score를 구하기 위해 일시적으로 불러왔을 뿐 예측과 상관 없으므로.

data.drop(['mean', 'std'], axis=1, inplace=True)

In [49]:
category_info = data.groupby(['cc_num', 'category']).agg(['mean', 'std'])['amt'].reset_index()

In [50]:
# data merging
data = data.merge(category_info, on = ['cc_num', 'category'], how = 'left')

In [51]:
data['cat_z_score'] = (data['amt'] - data['mean']) / data['std']
data.drop(['mean', 'std'], axis=1, inplace=True)

2) 거리: geopy 라이브러리를 사용해 계산 

In [52]:
import geopy.distance

In [53]:
# 위도와 경도를 한 변수로 통합
data['merch_coord'] = pd.Series(zip(data['merch_lat'], data['merch_long']))
data['cust_coord'] = pd.Series(zip(data['lat'], data['long']))

In [54]:
data['distance'] = data.apply(lambda x: geopy.distance.distance(x['merch_coord'], x['cust_coord']).km, axis=1)

In [55]:
# 이전과 같은 방식으로 z score 구함 
distance_info = data.groupby('cc_num').agg(['mean', 'std'])['distance'].reset_index()
data = data.merge(distance_info, on='cc_num', how='left')
data['distance_z_score'] = (data['distance'] - data['mean']) / data['std']
data.drop(['mean', 'std'], axis=1, inplace=True)

In [56]:
data.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,trans_data_trans_time,amt_z_score,cat_z_score,merch_coord,cust_coord,distance,distance_z_score
0,2019-01-01 00:00:18,2703186189652095,misc_net,4.97,F,36.0788,-81.1781,3495,1988-03-09,36.011293,-82.048315,0,2019-01-01 00:00:18,-0.662108,-0.688297,"(36.011293, -82.048315)","(36.0788, -81.1781)",78.773821,0.030974
1,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,2019-01-01 00:00:44,0.321302,0.317631,"(49.159047, -118.186462)","(48.8878, -118.2105)",30.216618,-1.475224
2,2019-01-01 00:00:51,38859492057661,entertainment,220.11,M,42.1808,-112.2620,4154,1962-01-19,43.150704,-112.154481,0,2019-01-01 00:00:51,1.287064,2.872509,"(43.150704, -112.154481)","(42.1808, -112.262)",108.102912,1.160572
3,2019-01-01 00:01:16,3534093764340240,gas_transport,45.00,M,46.2306,-112.1138,1939,1967-01-12,47.034331,-112.561071,0,2019-01-01 00:01:16,-0.125287,-1.050197,"(47.034331, -112.561071)","(46.2306, -112.1138)",95.685115,0.818063
4,2019-01-01 00:03:06,375534208663984,misc_pos,41.96,M,38.4207,-79.4629,99,1986-03-28,38.674999,-78.632459,0,2019-01-01 00:03:06,-0.565941,1.312866,"(38.674999, -78.632459)","(38.4207, -79.4629)",77.702395,0.064037


3) 나이: 생년월일을 활용해 나이를 계산(pandas의 dt 함수를 사용해 연월일 분리) 

In [57]:
# 나이: 우선은 예제와 동일하게 2021년으로 계산 적용 

data['age'] = 2021 - pd.to_datetime(data['dob']).dt.year 

In [58]:
# 예측에 불필요한 변수 제거

data.drop(['cc_num', 'lat', 'long', 'merch_lat', 'merch_long', 'dob', 'merch_coord', 'cust_coord'], axis=1, inplace=True)

In [59]:
data.head()

,trans_date_trans_time,category,amt,gender,city_pop,is_fraud,trans_data_trans_time,amt_z_score,cat_z_score,distance,distance_z_score,age
0,2019-01-01 00:00:18,misc_net,4.97,F,3495,0,2019-01-01 00:00:18,-0.662108,-0.688297,78.773821,0.030974,33
1,2019-01-01 00:00:44,grocery_pos,107.23,F,149,0,2019-01-01 00:00:44,0.321302,0.317631,30.216618,-1.475224,43
2,2019-01-01 00:00:51,entertainment,220.11,M,4154,0,2019-01-01 00:00:51,1.287064,2.872509,108.102912,1.160572,59
3,2019-01-01 00:01:16,gas_transport,45.00,M,1939,0,2019-01-01 00:01:16,-0.125287,-1.050197,95.685115,0.818063,54
4,2019-01-01 00:03:06,misc_pos,41.96,M,99,0,2019-01-01 00:03:06,-0.565941,1.312866,77.702395,0.064037,35


4) 더미 변수 변환: category, gender

In [60]:
data = pd.get_dummies(data, columns = ['category', 'gender'], drop_first=True)

5) 앞에서 datetime형으로 변경해 준 `trans_date_trans_time` 변수를 index로 보냄 

In [61]:
data.set_index('trans_date_trans_time', inplace=True)
data.head()

,amt,city_pop,is_fraud,trans_data_trans_time,amt_z_score,cat_z_score,distance,distance_z_score,age,category_food_dining,...,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M
trans_date_trans_time,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:18,4.97,3495,0,2019-01-01 00:00:18,-0.662108,-0.688297,78.773821,0.030974,33,0,...,0,0,0,1,0,0,0,0,0,0
2019-01-01 00:00:44,107.23,149,0,2019-01-01 00:00:44,0.321302,0.317631,30.216618,-1.475224,43,0,...,0,0,0,0,0,0,0,0,0,0
2019-01-01 00:00:51,220.11,4154,0,2019-01-01 00:00:51,1.287064,2.872509,108.102912,1.160572,59,0,...,0,0,0,0,0,0,0,0,0,1
2019-01-01 00:01:16,45.00,1939,0,2019-01-01 00:01:16,-0.125287,-1.050197,95.685115,0.818063,54,0,...,0,0,0,0,0,0,0,0,0,1
2019-01-01 00:03:06,41.96,99,0,2019-01-01 00:03:06,-0.565941,1.312866,77.702395,0.064037,35,0,...,0,0,0,0,1,0,0,0,0,1


4. modeling

시계열 데이터이므로 train_test_split이 아니라 특정 날짜를 기준으로 train/test를 split 해야 함 

In [62]:
train = data[data.index < '2020-07-01']
test = data[data.index >= '2020-07-01']

In [63]:
# 비율 확인

len(test) / len(data), len(train) / len(data)

(0.2837738623640543, 0.7162261376359457)

In [64]:
#target column 분리

X_train = train.drop('is_fraud', axis=1)
X_test = test.drop('is_fraud', axis=1)
y_train = train['is_fraud']
y_test = test['is_fraud']

In [67]:
X_train.head()

,amt,city_pop,trans_data_trans_time,amt_z_score,cat_z_score,distance,distance_z_score,age,category_food_dining,category_gas_transport,...,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M
trans_date_trans_time,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:18,4.97,3495,2019-01-01 00:00:18,-0.662108,-0.688297,78.773821,0.030974,33,0,0,...,0,0,0,1,0,0,0,0,0,0
2019-01-01 00:00:44,107.23,149,2019-01-01 00:00:44,0.321302,0.317631,30.216618,-1.475224,43,0,0,...,0,0,0,0,0,0,0,0,0,0
2019-01-01 00:00:51,220.11,4154,2019-01-01 00:00:51,1.287064,2.872509,108.102912,1.160572,59,0,0,...,0,0,0,0,0,0,0,0,0,1
2019-01-01 00:01:16,45.00,1939,2019-01-01 00:01:16,-0.125287,-1.050197,95.685115,0.818063,54,0,1,...,0,0,0,0,0,0,0,0,0,1
2019-01-01 00:03:06,41.96,99,2019-01-01 00:03:06,-0.565941,1.312866,77.702395,0.064037,35,0,0,...,0,0,0,0,1,0,0,0,0,1


4-1. 모델 로드

In [65]:
import lightgbm as lgb

In [66]:
# 1. 기본값으로 분류 함수를 이용해 모델링(하이퍼파라미터 튜닝 X)

model_1 = lgb.LGBMClassifier(random_state = 42) 
model_1.fit(X_train, y_train)
pred_1 = model_1.predict(X_test)

ValueError: ignored

4-2. 모델 평가 

In [34]:
# 평가를 위해 metrics load 

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

In [ ]:
# 1. accuracy score

accuracy_score(y_test, pred_1)

In [ ]:
# 2. confusion matrix 

print(confusion_matrix(y_test, pred_1))

In [ ]:
# 3. classification_report 

print(classification_report(y_test, pred_1))